# Install and import lib

In [ ]:
%pip install tweety-ns
%pip install https://github.com/mahrtayyab/tweety/archive/main.zip --upgrade 

In [5]:
from tweety import Twitter
import asyncio
import os
import csv
import tweety
import time


# Login


In [6]:

auth_token = os.getenv("auth_token1")

# Cookies can be a str or a dict

app = Twitter("session")
app.load_auth_token(auth_token)

<coroutine object AuthMethods.load_auth_token at 0x117f323b0>

In [7]:
async def login():
    # Read username and password from environment variables
    username = os.getenv("USERNAMEE")
    password = os.getenv("PASSWORD")

    if not username or not password:
        raise ValueError("USERNAMEE or PASSWORD is not configured in the .env file")

    # Initialize Twitter client
    app = Twitter("session")
    await app.sign_in(username, password)
    return app

In [8]:
app = await login()

# Crawl tweet


In [ ]:
async def crawl_tweets_from_KOLs(input_file_path, output_file_path, app, pages=1, replies=False, wait_time=2, cursor=None):
    tweets_data = []

    # Read the input file and get the list of screen names
    with open(input_file_path, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        screen_names = [row['screen_name'] for row in reader]
        screen_names = screen_names[140:210] #thay số chỗ nà

    # Open the error file to log users with rate limit errors
    with open("crawl_user_tweets_error.txt", mode="a", encoding="utf-8") as error_file:
        # Loop through each screen name to fetch tweets
        for screen_name in screen_names:
            print(f"Fetching tweets for user: {screen_name}")
            count = 0
            while True :
                if count > 3: 
                    print(f'skip {screen_name}')
                    break
                try:
                    all_tweets = await app.get_tweets(
                        username=screen_name,
                        pages=pages,
                        replies=replies,
                        wait_time=wait_time,
                        cursor=cursor
                    )
                    for tweet in all_tweets:
                        crawl = 0
                        try :
                            if type(tweet) == tweety.types.twDataTypes.Tweet:
                                crawl +=1
                                tweets_data.append({
                                    'id': tweet.id,
                                    'created_on': tweet.created_on,
                                    'date': tweet.date,
                                    'text': tweet.text,
                                    'rich_text': tweet.rich_text.text if tweet.rich_text else None,
                                    'author': tweet.author.username if tweet.author else None,
                                    'is_retweet': tweet.is_retweet,
                                    'retweeted_tweet_id': tweet.retweeted_tweet.id if tweet.is_retweet else None,
                                    'is_quoted': tweet.is_quoted,
                                    'quoted_tweet_id': tweet.quoted_tweet.id if tweet.is_quoted else None,
                                    'is_reply': tweet.is_reply,
                                    'replied_to': tweet.replied_to.id if tweet.is_reply else None,
                                    'is_sensitive': tweet.is_sensitive,
                                    'reply_counts': tweet.reply_counts,
                                    'quote_counts': tweet.quote_counts,
                                    'bookmark_count': tweet.bookmark_count,
                                    'views': tweet.views,
                                    'likes': tweet.likes,
                                    'language': tweet.language,
                                    'place': tweet.place.name if tweet.place else None,
                                    'retweet_counts': tweet.retweet_counts,
                                    'source': tweet.source,
                                    'has_moderated_replies': tweet.has_moderated_replies,
                                    'is_liked': tweet.is_liked,
                                    'is_retweeted': tweet.is_retweeted,
                                    'can_reply': tweet.can_reply,
                                    'broadcast': tweet.broadcast.title if tweet.broadcast else None,
                                    'edit_control': tweet.edit_control,
                                    'has_newer_version': tweet.has_newer_version,
                                    'audio_space_id': tweet.audio_space_id,
                                    'pool': tweet.pool.title if tweet.pool else None,
                                    'community': tweet.community.name if tweet.community else None,
                                    'media': [media.url for media in tweet.media],
                                    'user_mentions': [mention.username for mention in tweet.user_mentions],
                                    'urls': [url.url for url in tweet.urls],
                                    'hashtags': [hashtag.text for hashtag in tweet.hashtags],
                                    'symbols': [symbol.text for symbol in tweet.symbols],
                                    'community_note': tweet.community_note,
                                    'url': tweet.url,
                                    'threads': [thread.id for thread in tweet.threads],
                                    'comments': [comment.id for comment in tweet.comments]
                                })
                            elif type(tweet) == tweety.types.twDataTypes.SelfThread :
                                for sub_tweet in tweet.tweets :
                                    crawl +=1
                                    tweets_data.append({
                                'id': sub_tweet.id,
                                'created_on': sub_tweet.created_on,
                                'date': sub_tweet.date,
                                'text': sub_tweet.text,
                                'rich_text': sub_tweet.rich_text.text if sub_tweet.rich_text else None,
                                'author': sub_tweet.author.username if sub_tweet.author else None,
                                'is_retweet': sub_tweet.is_retweet,
                                'retweeted_tweet_id': sub_tweet.retweeted_tweet.id if sub_tweet.is_retweet else None,
                                'is_quoted': sub_tweet.is_quoted,
                                'quoted_tweet_id': sub_tweet.quoted_tweet.id if sub_tweet.is_quoted else None,
                                'is_reply': sub_tweet.is_reply,
                                'replied_to': sub_tweet.replied_to.id if sub_tweet.is_reply else None,
                                'is_sensitive': sub_tweet.is_sensitive,
                                'reply_counts': sub_tweet.reply_counts,
                                'quote_counts': sub_tweet.quote_counts,
                                'bookmark_count': sub_tweet.bookmark_count,
                                'views': sub_tweet.views,
                                'likes': sub_tweet.likes,
                                'language': sub_tweet.language,
                                'place': sub_tweet.place.name if sub_tweet.place else None,
                                'retweet_counts': sub_tweet.retweet_counts,
                                'source': sub_tweet.source,
                                'has_moderated_replies': sub_tweet.has_moderated_replies,
                                'is_liked': sub_tweet.is_liked,
                                'is_retweeted': sub_tweet.is_retweeted,
                                'can_reply': sub_tweet.can_reply,
                                'broadcast': sub_tweet.broadcast.title if sub_tweet.broadcast else None,
                                'edit_control': sub_tweet.edit_control,
                                'has_newer_version': sub_tweet.has_newer_version,
                                'audio_space_id': sub_tweet.audio_space_id,
                                'pool': sub_tweet.pool.title if sub_tweet.pool else None,
                                'community': sub_tweet.community.name if sub_tweet.community else None,
                                'media': [media.url for media in sub_tweet.media],
                                'user_mentions': [mention.username for mention in sub_tweet.user_mentions],
                                'urls': [url.url for url in sub_tweet.urls],
                                'hashtags': [hashtag.text for hashtag in sub_tweet.hashtags],
                                'symbols': [symbol.text for symbol in sub_tweet.symbols],
                                'community_note': sub_tweet.community_note,
                                'url': sub_tweet.url,
                                'threads': [thread.id for thread in sub_tweet.threads],
                                'comments': [comment.id for comment in sub_tweet.comments]
                            })
                        except : 
                            print('skip tweet')
                        print(f"Successfully fetched {crawl} tweets for {tweet}.")
                    break
                except Exception as e:
                    print(f"Error fetching tweets for user {screen_name}: {e}")
                    count += 1
                    time.sleep(60) #retry after 60 seconds

        # Check if the output file exists
        file_exists = os.path.isfile(output_file_path)

        # Save the results to the CSV file (append if the file exists, otherwise write header)
        with open(output_file_path, mode='a' if file_exists else 'w', newline='', encoding='utf-8') as outfile:
            fieldnames = [
                'id', 'created_on', 'date', 'text', 'rich_text', 'author', 'is_retweet', 'retweeted_tweet_id',
                'is_quoted', 'quoted_tweet_id', 'is_reply', 'replied_to', 'is_sensitive', 'reply_counts',
                'quote_counts', 'bookmark_count', 'views', 'likes', 'language', 'place', 'retweet_counts',
                'source', 'has_moderated_replies', 'is_liked', 'is_retweeted', 'can_reply', 'broadcast',
                'edit_control', 'has_newer_version', 'audio_space_id', 'pool', 'community', 'media', 
                'user_mentions', 'urls', 'hashtags', 'symbols', 'community_note', 'url', 'threads', 'comments'
            ]
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)

            # Write header only if the file does not exist
            if not file_exists:
                writer.writeheader()

            writer.writerows(tweets_data)

        print(f"Saved {len(tweets_data)} tweets to {output_file_path}.")


In [10]:
# # Basic Tweet Information
# 'id': int                # The unique ID of the tweet (assigned by Twitter).
# 'created_on': datetime   # The date and time when the tweet was created.
# 'date': datetime         # The date and time when the tweet was created (same as created_on).

# # Main Content of the Tweet
# 'text': str              # The text content of the tweet (original text).
# 'rich_text': RichText    # The rich text content (if any), which may include HTML formatting or other styles.
# 'author': User           # The user object containing information about the tweet's author (e.g., username, ID, etc.).

# # Tweet Interaction (Retweets, Quotes, Replies)
# 'is_retweet': bool              # If `True`, the tweet is a retweet of another tweet.
# 'retweeted_tweet_id': int       # The ID of the original tweet that was retweeted (if applicable).
# 'is_quoted': bool               # If `True`, the tweet is a quoted tweet.
# 'quoted_tweet_id': int          # The ID of the tweet that was quoted (if applicable).
# 'is_reply': bool                # If `True`, the tweet is a reply to another tweet.
# 'replied_to': Tweet | str       # The tweet being replied to (if applicable), or the user ID if it is a reply to a user.

# # Sensitive Content Information
# 'is_sensitive': bool            # If `True`, the tweet contains sensitive content (e.g., graphic images, NSFW).

# # Engagement Metrics
# 'reply_counts': int            # The number of times someone has replied to this tweet.
# 'quote_counts': int            # The number of times this tweet has been quoted.
# 'bookmark_count': int          # The number of times this tweet has been bookmarked.
# 'views': int                   # The number of times this tweet has been viewed.
# 'likes': int                   # The number of likes (favorites) this tweet has received.
# 'language': str                # The language of the tweet as identified by Twitter (e.g., 'en' for English).
# 'place': Place                 # The location or place associated with the tweet, if available.

# # Retweet Information
# 'retweet_counts': int          # The number of times this tweet has been retweeted.

# # Source and Moderation Information
# 'source': str                  # The source application used to post the tweet (e.g., "Twitter Web App").
# 'has_moderated_replies': bool  # If `True`, replies to the tweet are moderated by the author.

# # User Interaction Information
# 'is_liked': bool               # If `True`, the authenticated user has liked this tweet.
# 'is_retweeted': bool           # If `True`, the authenticated user has retweeted this tweet.
# 'can_reply': bool              # If `True`, the authenticated user is allowed to reply to this tweet.

# # Broadcast and Edit Information
# 'broadcast': Broadcast | None  # Information about the broadcast (if the tweet is part of a broadcast).
# 'edit_control': EditControl | None  # Information about the tweet's edit control (if it has been edited).
# 'has_newer_version': bool      # If `True`, this tweet has been edited, and there is a newer version of it.

# # Additional Tweet Metadata
# 'audio_space_id': str          # The ID of the Audio Space associated with the tweet (if applicable).
# 'pool': Pool | None            # The pool associated with the tweet (if applicable).
# 'community': Community | None  # The community this tweet is a part of (if applicable).

# # Media and Mentions
# 'media': list[Media]          # A list of media objects (e.g., images, videos) attached to the tweet.
# 'user_mentions': list[ShortUser]  # A list of users mentioned in the tweet.
# 'urls': list[URL]             # A list of URLs mentioned in the tweet.
# 'hashtags': list[Hashtag]      # A list of hashtags mentioned in the tweet.
# 'symbols': list[Symbol]       # A list of symbols (e.g., stock symbols) mentioned in the tweet.

# # Community and Thread Information
# 'community_note': str | None   # A community note posted in response to the tweet (if applicable).
# 'url': str                     # The URL of the tweet.
# 'threads': list[Tweet]         # A list of tweets in the thread of this tweet.
# 'comments': list[ConversationThread]  # A list of replies or comments sent in response to this tweet.


In [ ]:

input_file_path = "cleanedd_KOL_profile_status_count_more_than_200.csv"
output_file_path ="final_tweet.csv"
await crawl_tweets_from_KOLs(input_file_path, output_file_path, app, pages=10, replies=False, wait_time=10, cursor=None)

Fetching tweets for user: OffTheChainGang
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
Fetching tweets for user: Official_ARize
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
Fetching tweets for user: OmniFlixNetwork
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
Fetching tweets for user: Ordinary_Adam
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet
skip tweet

In [9]:
# Read keywords from a text file
def read_keywords_from_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return [line.strip() for line in file if line.strip()]
# Danh sách các giá trị trong cột 'b' mà bạn muốn xóa
X = read_keywords_from_file("loc.txt")
print(X)

['1281273004341841923', '1498984512', '20486941', '1064986661723922432', '1264467031384088581', '1487057201328009219', '1159451278008901632', '1496174374075916294', '1563771264', '1303420675114201089', '1453573933815369731', '1431050544685088768', '885490321550868480', '1248332026790330369', '1619659433952546817', '1462451461728587780', '1301509419696558082', '791786474', '1392088851221413893', '1457015038691057666', '1416665472469135365', '1376655684872200192', '43358536', '262165644', '857031238569181184', '69777546', '1384010537328025602', '1716071804853268481', '1353693148321624066', '2438767100', '263689859', '1545599597036605440', '2310943632', '23942310', '70355577', '1530371', '812505876754038784', '1416471276831469569', '851104049722122240', '961335348474281987', '944510856712224770', '67197023', '1455391184411496458', '2370753746', '9897812', '857729198198095873', '14196323', '45360564', '17225803', '1326397615202820096', '1478555106652086280', '197034892', '48036658', '10539

In [10]:
import pandas as pd
df = pd.read_csv("cleanedd_KOL_profile_status_count_more_than_200.csv")
df_cleaned = df[~df['id'].isin(X)]
print(df['id'].dtype)

int64
